<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Lesson14_Features/AI_DimensionReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема семинара: отбор признаков

- Фильтрационные методы
- Оберточные методы
- Встроенные методы
- Метод главных компонент или PCA

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/ML_AI24/refs/heads/main/Lesson14_Features/Pokemon.csv')

In [ ]:
data

#                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0         65       65     45           1      False  
1         80       80     60           1      False  
2        100      100     80           1      False  
3        122      120     80           1      False  
4         60       50     65           1      False  
..       ...      ...    ...         ...        ...  
795      100      150     50           6       True  
796      160      110    110           6       True  
797      150      130     70           6       True  
798      170      130     80           6       True  
799      130       90     70           6       True  

[800 rows x 13 columns]

Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [ ]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

data.drop(columns=['#', 'Name'], inplace=True)

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [ ]:
X = data.drop(columns='Legendary')
y = data['Legendary'].astype('int')

In [ ]:
y.value_counts(normalize=True)

0    0.91875
1    0.08125
Name: Legendary, dtype: float64

# Make some default pipeline

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [ ]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
# cross_val_score(default_pipeline,
#                         X,
#                         y,
#                         cv=5,
#                         scoring='f1'
#                        ).mean()

In [ ]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

In [ ]:
cv_res1['test_score'].mean()

0.5466128466128467

# Make pipeline more complicated

In [ ]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

cv_res2

{'fit_time': array([0.21799898, 0.24099946, 0.23699594, 0.24100089, 0.2399888 ]),
 'score_time': array([0.02699924, 0.02000356, 0.01800179, 0.01699924, 0.01899695]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [ ]:
cv_res2['train_score'].mean()

0.9522051815498418

In [ ]:
cv_res2['test_score'].mean()

0.5934281842818427

train_score - просто класс ! модель получилась сложная, только очевидно переобученная ...

согласны, узнали ?


# Introduce feature selectors

In [ ]:
data_tr = pipe_dif[:-1]

In [ ]:
data_tr

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler())])

In [ ]:
X_tr = data_tr.fit_transform(X, y)
print(f'data shape after transformation is {X_tr.shape}')

data shape after transformation is (800, 1001)


c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

В том числе на лекции обсуждались:

 - статистика хи-квадрат
 - метрика mutual information

In [ ]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [ ]:
# k_best = 30

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func=mutual_info_classif, k=100)),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
cv_res = cross_validate(pipe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([4.21170735, 4.41992903, 4.33466315, 4.15234232, 3.92775798]),
 'score_time': array([0.01000094, 0.02095604, 0.01303077, 0.01396418, 0.01099801]),
 'test_score': array([0.26666667, 0.60869565, 0.62068966, 0.56      , 0.60606061]),
 'train_score': array([0.83168317, 0.66666667, 0.81188119, 0.79166667, 0.86      ])}

In [ ]:
# k best нужно подбирать

cv_res['test_score'].mean()

0.5324225160147199

In [ ]:
cv_res['train_score'].mean()

0.7923795379537953

## Жадный метод отбора

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
k_best = 50

rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [ ]:
X_tr.shape

(800, 1001)

In [ ]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

(800, 50)

array([[-0.98555744, -0.53089061, -0.93734374, ..., -0.40535388,
        -0.52135831, -0.81966779],
       [-0.48479877, -0.53089061, -0.84621022, ..., -0.16466941,
        -0.18003271, -0.81698899],
       [ 0.42451538, -0.53089061, -0.68419506, ...,  0.45023305,
         0.73602435, -0.81341726],
       ...,
       [-0.16049792, -0.53089061,  1.16740671, ...,  3.97831918,
         2.1076934 ,  4.18343684],
       [-0.16049792, -0.53089061,  1.16740671, ...,  6.06069716,
         2.51608978,  4.18343684],
       [ 1.36562373, -0.53089061,  1.16740671, ...,  1.4680924 ,
         0.19750623,  2.6404483 ]])

In [ ]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter=1000),
                      n_features_to_select=20,
                      step=30
                     )),
    ('model_', SVC(kernel='linear'))])

In [ ]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([4.36161995, 5.94719887, 4.55667353, 4.71356058, 4.96263337]),
 'score_time': array([0.01299763, 0.01900053, 0.03300047, 0.02998376, 0.01599908]),
 'test_score': array([0.35294118, 0.86956522, 0.56      , 0.58333333, 0.64864865]),
 'train_score': array([0.82474227, 0.84      , 0.86      , 0.84210526, 0.88659794])}

In [ ]:
cv_res3['test_score'].mean()

0.6028976751687749

In [ ]:
cv_res3['train_score'].mean()

0.8506890938686924

## С помощью модели

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
sel = SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=0.1)

In [ ]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

(800, 37)

array([[-0.7732015 , -0.80122124, -0.31795653, ..., -0.52135831,
        -0.72668962, -0.81966779],
       [-0.33767384, -0.35246512, -0.30715816, ..., -0.18003271,
        -0.70651232, -0.81698899],
       [ 0.6523422 ,  0.66762108, -0.28796106, ...,  0.73602435,
        -0.6731154 , -0.81341726],
       ...,
       [ 0.80566326,  0.55477968,  0.91906366, ...,  2.1076934 ,
         4.87819432,  4.18343684],
       [ 1.06849938,  0.55477968,  0.91906366, ...,  2.51608978,
         4.87819432,  4.18343684],
       [ 2.2512619 ,  1.23182804, -0.18905176, ...,  0.19750623,
         1.93926564,  2.6404483 ]])

In [ ]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear'),
                                  threshold=1e-5)),
    ('model_', SVC(kernel='linear'))])

In [ ]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res4

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([0.14643931, 0.15901089, 0.11522102, 0.1359756 , 0.13002992]),
 'score_time': array([0.01403737, 0.01100016, 0.01000071, 0.01399827, 0.0110023 ]),
 'test_score': array([0.44444444, 0.84615385, 0.64285714, 0.60869565, 0.68421053]),
 'train_score': array([0.92307692, 0.89795918, 0.95145631, 0.93877551, 0.92929293])}

In [ ]:
cv_res4['test_score'].mean()

0.6452723223890271

# PCA или метод главных компонент

Цель: создать k новых признаков из какого-либо количества старых признаков, так чтобы
- каждый из новых признаков был линейной комбинацией старых

$z_i = u_1x_{1i} + ... + u_lx{li}$

- и дисперсия $z_i$, то есть новых признаков была максимальной (наиболее информативной)

С точки зрения линеной алгебры, процесс нахождения новых признаков из старых - это процесс проекции старых признаков на некоторую гиперплоскоть (линейное пространство). Как было показано на лекции, базисом этого пространства являются собственные вектора матрицы $X^TX$ - где Х - это центрированная матрица признаков

Тогда чтобы найти новые признаки (главные компоненты) нужно сначала
- найти собственные вектора V матрицы $X^TX$ (вектора должны быть приведены к длине 1)
- произвести матричное умножение Z = XV (то есть сделать проекцию матрицы X на линейное пространство с базисом V)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

#X = [[1,3],[0,2],[0,0],[3,3]]
X = [[0,1],[-1,0],[-1,-2],[2,1]]
pca.fit_transform(X)

array([[-0.70710678,  0.70710678],
       [ 0.70710678,  0.70710678],
       [ 2.12132034, -0.70710678],
       [-2.12132034, -0.70710678]])

In [ ]:
pca.explained_variance_ratio_

array([0.83333333, 0.16666667])

In [ ]:
# пример

pca = PCA(n_components = 15)

In [ ]:
res = pca.fit_transform(X_tr)
display(res.shape)
res

(800, 15)

array([[-1.98007291e+01,  4.46072632e+00, -4.82046552e-01, ...,
         2.70485986e-02,  4.35339188e-01, -1.29268003e+00],
       [-1.58785523e+01,  3.10134435e+00, -3.17353060e+00, ...,
        -8.35833252e-01,  2.34488866e-01, -1.23767803e+00],
       [-6.24072086e+00, -4.63280109e-01, -9.93003245e+00, ...,
        -3.27924084e+00, -4.32771767e-01, -1.25641943e+00],
       ...,
       [ 3.26767745e+01,  1.50340452e+01, -2.46534010e+01, ...,
         5.00607056e+00, -1.42600789e-02,  3.51084334e+00],
       [ 5.53322261e+01,  1.48563726e+01, -3.86325556e+01, ...,
         5.42144504e+00, -2.40256784e+00,  4.45884714e-01],
       [ 1.99828053e+01, -4.60646431e+00, -2.56386391e+01, ...,
         1.78605965e+00, -1.85612137e+00,  1.45795949e+00]])

In [ ]:
# суммарная доля объясненной дисперсии исходных признаков

pca.explained_variance_ratio_.sum()

0.9438803480955492

In [ ]:
# каждая следующая компонента менее информативна чем предыдущая
pca.explained_variance_ratio_

array([0.45670112, 0.1176569 , 0.08837368, 0.07259347, 0.05161779,
       0.03907593, 0.03147126, 0.02632701, 0.01579202, 0.01454208,
       0.00758009, 0.00692028, 0.00568022, 0.0051811 , 0.00436741])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_components = 15

pipe_pca = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', PCA(n_components=n_components)),
    ('model_', RandomForestClassifier())])

cv_res5 = cross_validate(pipe_pca, X, y, cv=5, scoring='roc_auc', return_train_score=True)
cv_res5

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([0.34381175, 0.30033875, 0.3590107 , 0.39999962, 0.35354018]),
 'score_time': array([0.02400303, 0.03196907, 0.02191544, 0.02499771, 0.02252722]),
 'test_score': array([0.93851387, 0.96912611, 0.95499738, 0.94453166, 0.95028781]),
 'train_score': array([0.99887166, 0.99856096, 0.99896978, 0.99888802, 0.99960754])}

In [ ]:
cv_res5['test_score'].mean()

0.9514913657770802

In [ ]:
cv_res5['train_score'].mean()

0.9989795918367346